In [54]:
import numpy as np
import pandas as pd
import pickle
from statsmodels.tsa.statespace.sarimax import SARIMAX

from utils_functions import *

## Functions

In [55]:
def sarimax_forecast(riderships_data):
    """Generate a SARIMAX forecast for the next month based on ridership data.

    This function fits a SARIMAX model to the provided ridership data and 
    forecasts values for the number of days in the next month.

    Args:
        riderships_data (pd.DataFrame): A DataFrame containing a column 
            'number_of_riderships' representing the ridership time series data.

    Returns:
        pd.DataFrame: A DataFrame containing the forecasted values for the next month.
            Columns:
            - 'predicted_mean': Forecasted mean values.
            - 'lower_bound': Lower bound of the confidence interval.
            - 'upper_bound': Upper bound of the confidence interval.
    """
    all_data = riderships_data['number_of_riderships']

    sarimax_model = SARIMAX(all_data, order=(1, 1, 1), seasonal_order=(1, 0, 0, 7))
    sarimax_fitted = sarimax_model.fit(disp=False)

    forecast_steps = days_in_next_month()
    forecast_obj = sarimax_fitted.get_forecast(steps=forecast_steps)
    forecast_mean = forecast_obj.predicted_mean
    conf_int = forecast_obj.conf_int(alpha=0.4)

    forecasts = pd.DataFrame({
        'predicted_mean': forecast_mean,
        'lower_bound': conf_int.iloc[:, 0],
        'upper_bound': conf_int.iloc[:, 1]
    })

    return forecasts

In [56]:
TOP_20_STATIONS = [611, 610, 607, 602, 628, 164, 614, 318, 624, 613, 612, 225, 618, 609, 601, 397, 623, 167, 619, 313]

## Import Data

In [57]:
file_key = 'data-transformed/run-1731935541465-part-r-00000.csv'
mta_subway_df = read_s3_csv_to_dataframe(file_key)
mta_subway_df = mta_subway_df.sort_values(by=["station_complex_id", "created_date"], ascending=True)
mta_subway_df.set_index('created_date', inplace=True)
mta_subway_by_station = mta_subway_df[['station_complex_id', 'number_of_riderships']]
mta_subway_by_station = mta_subway_by_station[mta_subway_by_station.station_complex_id.isin(TOP_20_STATIONS)]

In [58]:
mta_subway_df.head()

,station_complex_id,station_complex,latitude,longitude,georeference,number_of_riderships,year_period
created_date,,,,,,,
2023-01-01,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),8909.0,2023
2023-01-02,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),7475.0,2023
2023-01-03,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),9291.0,2023
2023-01-04,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),10697.0,2023
2023-01-05,8,"5 Av/59 St (N,R,W)",40.764812,-73.97335,POINT (-73.97335 40.764812),10391.0,2023


In [59]:
mta_subway_by_station.head()

,station_complex_id,number_of_riderships
created_date,,
2023-01-01,164,28229.0
2023-01-02,164,30886.0
2023-01-03,164,50083.0
2023-01-04,164,51473.0
2023-01-05,164,51049.0


In [60]:
mta_subway_by_station.station_complex_id.unique()

array([164, 167, 225, 313, 318, 397, 601, 602, 607, 609, 610, 611, 612,
       613, 614, 618, 619, 623, 624, 628])

## Make Predictions to all stations

In [61]:
forecasts = mta_subway_by_station.groupby('station_complex_id').apply(sarimax_forecast)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/statsmodels/tsa/s

In [62]:
forecasts = forecasts.reset_index()
forecasts.columns = ['station_complex_id', 'created_date', 'predicted_mean', 'lower_bound', 'upper_bound']
forecasts.set_index('created_date', inplace=True)
data_and_predictions = pd.concat([mta_subway_by_station, forecasts], axis=0)
mta_subway_df_unique = mta_subway_df.reset_index()[["station_complex_id", "station_complex", "latitude", "longitude"]].drop_duplicates(subset=["station_complex_id"])
final_data_to_save = pd.merge(data_and_predictions.reset_index(), mta_subway_df_unique, on="station_complex_id", how="left")

In [63]:
final_data_to_save.tail()

,created_date,station_complex_id,number_of_riderships,predicted_mean,lower_bound,upper_bound,station_complex,latitude,longitude
13395,2024-10-27,628,NaN,35889.576187,25362.045319,46417.107055,"Fulton St (A,C,J,Z,2,3,4,5)",40.710373,-74.00769
13396,2024-10-28,628,NaN,36664.459951,26136.902544,47192.017358,"Fulton St (A,C,J,Z,2,3,4,5)",40.710373,-74.00769
13397,2024-10-29,628,NaN,50458.265668,39496.461373,61420.069963,"Fulton St (A,C,J,Z,2,3,4,5)",40.710373,-74.00769
13398,2024-10-30,628,NaN,53547.070173,42531.213538,64562.926807,"Fulton St (A,C,J,Z,2,3,4,5)",40.710373,-74.00769
13399,2024-10-31,628,NaN,53503.858977,42480.490082,64527.227871,"Fulton St (A,C,J,Z,2,3,4,5)",40.710373,-74.00769


## Save Predictions

In [64]:
save_dataframe_to_s3(final_data_to_save, bucket_name, 'predictions/riderships_predictions.csv')

DataFrame successfully saved to s3://mta-subway/predictions/riderships_predictions.csv
